In [16]:
import cPickle
from PIL import Image
import os
import sys
import numpy as np
from matplotlib.pyplot import imshow
import tensorflow as tf
import random
import datetime


train_tfrecord_filenames = []
test_tfrecord_filenames = []

train_tfrecord_folder = 'tfrecords/train_tfrecords'
test_tfrecord_folder = 'tfrecords/test_tfrecords'

for filename in os.listdir(train_tfrecord_folder):
    train_tfrecord_filenames.append(os.path.join(train_tfrecord_folder, filename))
for filename in os.listdir(test_tfrecord_folder):
    test_tfrecord_filenames.append(os.path.join(test_tfrecord_folder, filename))

print len(train_tfrecord_filenames)
print train_tfrecord_filenames[0:3]
print len(test_tfrecord_filenames)
print test_tfrecord_filenames[0:3]
        

50
['tfrecords/train_tfrecords/tfrecord-0', 'tfrecords/train_tfrecords/tfrecord-1', 'tfrecords/train_tfrecords/tfrecord-10']
10
['tfrecords/test_tfrecords/tfrecord-0', 'tfrecords/test_tfrecords/tfrecord-1', 'tfrecords/test_tfrecords/tfrecord-2']


In [17]:
filename_placeholder = tf.placeholder(tf.string, shape=[None])

def _parse_function(example_proto):
    features = {"img_raw": tf.FixedLenFeature((), tf.string, default_value=""),
                "label": tf.FixedLenFeature((), tf.int64, default_value=0)}
    parsed_features = tf.parse_single_example(example_proto, features)
    img_decoded = tf.decode_raw(parsed_features['img_raw'], tf.uint8)
    return img_decoded, parsed_features['label']

dataset = tf.data.TFRecordDataset(filename_placeholder)
dataset = dataset.map(_parse_function)

batch_size = 20
dataset = dataset.prefetch(buffer_size=2000)
dataset = dataset.batch(batch_size)
dataset = dataset.shuffle(buffer_size=2000)
dataset = dataset.repeat()
# iterator = dataset.make_one_shot_iterator()
iterator = dataset.make_initializable_iterator()
batch_imgs, batch_labels = iterator.get_next()
batch_imgs = tf.cast(batch_imgs, tf.float32)
batch_labels = tf.cast(batch_labels, tf.int64)


In [18]:
def cnn_model(x, y):
    with tf.name_scope("inputs"):
        is_training = tf.placeholder(tf.bool, name="is_training")
        x_image = tf.reshape(x, [-1, 32, 32, 3])
    with tf.name_scope("model"):
        conv1 = tf.layers.conv2d(inputs=x_image,
                                 filters=32,
                                 kernel_size=[5,5],
                                 padding='same',
                                 activation=tf.nn.relu)
        pool1 = tf.layers.max_pooling2d(inputs=conv1,
                                        pool_size=[2,2],
                                        strides=2)
        
        conv2 = tf.layers.conv2d(inputs=pool1,
                                 filters=64,
                                 kernel_size=[5,5],
                                 padding='same',
                                 activation=tf.nn.relu)
        pool2 = tf.layers.max_pooling2d(inputs=conv2,
                                        pool_size=[2,2],
                                        strides=2)
        
        conv3 = tf.layers.conv2d(inputs=pool2,
                                 filters=64,
                                 kernel_size=[5,5],
                                 padding='same',
                                 activation=tf.nn.relu)
        pool3 = tf.layers.max_pooling2d(inputs=conv3,
                                        pool_size=[2,2],
                                        strides=2)
        
        pool3_flatten = tf.layers.flatten(pool3)
        dense = tf.layers.dense(inputs=pool3_flatten,
                                units=128,
                                activation=tf.nn.relu)
        dropout = tf.layers.dropout(inputs=dense,
                                    rate=0.1,
                                    training=is_training)
        logits = tf.layers.dense(inputs=dropout,
                                units=10)
    return is_training, logits

In [20]:
is_training, logits = cnn_model(batch_imgs, batch_labels)
with tf.name_scope('metrics'):
    cross_entropy = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(labels=batch_labels, logits=logits))
    correct_prediction = tf.equal(tf.argmax(logits, 1), batch_labels)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)

init = tf.global_variables_initializer()

train_steps = 500
with tf.Session() as sess:
    sess.run(init)
    sess.run(iterator.initializer,
             feed_dict = {
                 filename_placeholder: train_tfrecord_filenames,
             })
    starttime = datetime.datetime.now()
    for i in range(train_steps):
        cross_entropy_val, acc_val, _ = sess.run([cross_entropy, accuracy, train_op],
                                                 feed_dict = {
                                                     is_training: True
                                                 })

        if (i+1) % 50 == 0:
            endtime = datetime.datetime.now()
            timedelta = endtime - starttime
            print '[Train] Step %4d: ce: %4.5f, acc: %4.5f [Time: %d.%d s]' % (
                i+1, cross_entropy_val, acc_val,
                timedelta.seconds, timedelta.microseconds / 1000)
            starttime = datetime.datetime.now()

[Train] Step   50: ce: 2.19784, acc: 0.15000 [Time: 8.21 s]
[Train] Step  100: ce: 2.10620, acc: 0.15000 [Time: 5.13 s]
[Train] Step  150: ce: 1.73219, acc: 0.45000 [Time: 5.503 s]
[Train] Step  200: ce: 1.97300, acc: 0.25000 [Time: 5.432 s]
[Train] Step  250: ce: 1.77547, acc: 0.30000 [Time: 5.433 s]
[Train] Step  300: ce: 1.65656, acc: 0.35000 [Time: 5.173 s]
[Train] Step  350: ce: 1.77073, acc: 0.40000 [Time: 5.13 s]
[Train] Step  400: ce: 2.06213, acc: 0.30000 [Time: 5.125 s]
[Train] Step  450: ce: 1.50392, acc: 0.55000 [Time: 5.12 s]
[Train] Step  500: ce: 1.38159, acc: 0.50000 [Time: 5.91 s]
